<a href="https://colab.research.google.com/github/Aridel7/Study_2/blob/main/DLL_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Рекуррентные сети 2**

Задание 1

Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:

x - последовательность цифр

y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:

научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU

In [46]:
import torch
import time

In [7]:
X_1 = torch.randint(0,9,(5000,25))
X_2 = torch.randint(0,9,(5000,75))
X_3 = torch.randint(0,9,(5000,150))

In [33]:
def get_Y(X_n):
  Y = torch.zeros_like(X_n)
  for i, X in enumerate(X_n):
    for j, x in enumerate(X):
      if j == 0:
        Y[i][j] = x
      else:
        Y[i][j] = x + X[0]
        if Y[i][j] >= 10:
          Y[i][j] -= 10
  return Y

In [35]:
Y_1 = get_Y(X_1)
Y_2 = get_Y(X_3)
Y_3 = get_Y(X_3)

In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_1, Y_1, test_size=.2, random_state=13)
X_train = X_train.float()
X_test = X_test.float()
y_train = y_train.float()
y_test = y_test.float()

In [149]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()

        self.num_hiddens = num_hiddens
        self.embedding = torch.nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens)#, batch_first=True)
        self.output = torch.nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        #predictions = self.output(state[0].squeeze())
        predictions = self.output(_)
        return predictions

In [150]:
model_1 = NeuralNetwork(torch.nn.RNN, 10, 64, 128, 10)

In [151]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_1.parameters())

In [153]:
for ep in range(100):
    train_loss, train_acc, iter_num = .0, .0, .0
    start = time.time()

    model_1.train()
    for i in range(int(len(X_train) / 512)):

        batch_x = X_train[i * 512:(i + 1) * 512]
        batch_y = y_train[i * 512:(i + 1) * 512].flatten()

        optimizer.zero_grad()
        answers = model_1.forward(batch_x.to(torch.int64)).view(-1, 10)
        loss = criterion(answers, batch_y.to(torch.int64))
        train_loss += loss.item()

        batch_acc = (answers.argmax(dim=1) == batch_y)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]

        loss.backward()
        optimizer.step()
        iter_num += 1

    print(
        f"Epoch: {ep}, loss: {train_loss:.4f}, acc: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )

    test_loss, test_acc, iter_num = .0, .0, .0
    model_1.eval()
    for i in range(int(len(X_test) / 100)):

        batch_x = X_test[i * 100:(i + 1) * 100]
        batch_y = y_test[i * 100:(i + 1) * 100].flatten()

        answers = model_1.forward(batch_x.to(torch.int64)).view(-1, 10)
        loss = criterion(answers, batch_y.to(torch.int64))
        test_loss += loss.item()

        batch_acc = (answers.argmax(dim=1) == batch_y)
        test_acc += batch_acc.sum().item() / batch_acc.shape[0]

        iter_num += 1
    print(
        f"test loss: {test_loss:.4f}, test acc: {test_acc / iter_num:.4f} | "
        f"{time.time() - start:.2f} sec."
    )

Epoch: 0, loss: 15.6937, acc: 0.1253 | test loss: 22.2920, test acc: 0.1392 | 0.98 sec.
Epoch: 1, loss: 15.5639, acc: 0.1330 | test loss: 22.1634, test acc: 0.1496 | 0.96 sec.
Epoch: 2, loss: 15.4906, acc: 0.1368 | test loss: 22.0922, test acc: 0.1492 | 0.96 sec.
Epoch: 3, loss: 15.4480, acc: 0.1369 | test loss: 22.0531, test acc: 0.1492 | 0.95 sec.
Epoch: 4, loss: 15.4214, acc: 0.1369 | test loss: 22.0250, test acc: 0.1504 | 1.28 sec.
Epoch: 5, loss: 15.4039, acc: 0.1375 | test loss: 22.0073, test acc: 0.1504 | 1.45 sec.
Epoch: 6, loss: 15.3919, acc: 0.1372 | test loss: 21.9978, test acc: 0.1492 | 0.95 sec.
Epoch: 7, loss: 15.3831, acc: 0.1373 | test loss: 21.9898, test acc: 0.1494 | 0.94 sec.
Epoch: 8, loss: 15.3759, acc: 0.1376 | test loss: 21.9841, test acc: 0.1487 | 0.97 sec.
Epoch: 9, loss: 15.3697, acc: 0.1377 | test loss: 21.9804, test acc: 0.1486 | 0.98 sec.
Epoch: 10, loss: 15.3644, acc: 0.1379 | test loss: 21.9774, test acc: 0.1479 | 0.94 sec.
Epoch: 11, loss: 15.3597, acc: 

In [69]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model_1.train()
    for X_b, y_b in data_1:
        #X_b, y_b = X_b.cuda(), y_b.cuda()
        #X_b, y_b = X_b, y_b
        optimizer.zero_grad()
        answers = model_1(X_b.to(torch.int64)).view(-1, 10)
        loss = criterion(answers, y_b.to(torch.int64).reshape(-1,25))
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    #model_1.eval()
    #generate_text()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])